In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import math
from collections import OrderedDict

In [12]:
# Constant for calculation
charging_station_distance = 100

In [2]:
df = pd.read_csv('Trip Seg WK 1-4.csv')

/Users/m0s01q5/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (22,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(1009542, 29)

In [4]:
df.describe()

,trip_id,trip_seq,load_id,o_loc_id,d_loc_id,miles,dwell,week,year,o_lat,o_lon,d_lat,d_lon
count,1.009542e+06,1.009542e+06,1.009542e+06,1.009542e+06,1.009542e+06,1.007027e+06,675807.000000,1.009542e+06,1009542.0,1.009542e+06,1.009542e+06,1.009542e+06,1.009542e+06
mean,9.092689e+07,2.389822e+00,4.314166e+07,5.136594e+06,5.051856e+06,9.653761e+01,27.402421,2.511284e+00,2019.0,3.649243e+01,-9.129132e+01,3.649289e+01,-9.129221e+01
std,1.132646e+05,2.293130e+00,2.943718e+07,1.766492e+07,1.752721e+07,9.565400e+01,122.665696,1.118272e+00,0.0,4.797010e+00,1.309608e+01,4.797167e+00,1.309698e+01
min,9.029722e+07,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-7404.000000,1.000000e+00,2019.0,0.000000e+00,-1.242521e+02,0.000000e+00,-1.242521e+02
25%,9.083450e+07,1.000000e+00,0.000000e+00,2.262000e+03,2.261000e+03,2.800000e+01,28.000000,2.000000e+00,2019.0,3.298720e+01,-9.715070e+01,3.299160e+01,-9.715040e+01
50%,9.092762e+07,2.000000e+00,6.305205e+07,5.713000e+03,5.717000e+03,6.900000e+01,30.000000,3.000000e+00,2019.0,3.619180e+01,-8.827130e+01,3.619180e+01,-8.827130e+01
75%,9.102015e+07,3.000000e+00,6.333115e+07,6.732000e+03,6.697000e+03,1.360000e+02,30.000000,4.000000e+00,2019.0,4.041420e+01,-8.195000e+01,4.041420e+01,-8.194150e+01
max,9.123116e+07,7.300000e+01,6.386565e+07,9.999948e+07,9.999948e+07,1.114000e+03,180.000000,4.000000e+00,2019.0,4.898970e+01,1.171769e+02,4.898970e+01,1.171769e+02


In [5]:
df.apply(lambda x: sum(x.isnull()), 0)

trip_id                 0
bt_date                 0
trip_seq                0
load_id                 0
o_loc_id                0
o_loc_type_code         0
d_loc_id                0
d_loc_type_code         0
o_activity_ts        2306
o_ac_type_code          0
d_act_ts             2306
d_ac_type_code          0
ld_att             896987
miles                2515
dwell              333735
trip_type               0
week                    0
year                    0
o_lat                   0
o_lon                   0
o_city                  0
o_state                 0
o_loc_zip               4
d_lat                   0
d_lon                   0
d_city                  0
d_state                 0
d_loc_zip               4
miles_type              0
dtype: int64

In [6]:
df1 = df.dropna(axis=0, subset=['o_activity_ts'])
df1 = df1.dropna(axis =0, subset=['miles'])
df1 = df1.dropna(axis =0, subset=['o_loc_zip'])
df1 = df1.dropna(axis =0, subset=['d_loc_zip'])

In [7]:
import json
from collections import defaultdict


In [8]:
def get_route(
    o_loc_zip, o_lat, o_lon,
    d_loc_zip, d_lat, d_lon):
    
    global counter
    counter += 1
    print(counter)
    if (o_loc_zip, d_loc_zip) in dict_source_destination:
        return
    
    url_string = 'https://route.ls.hereapi.com/routing/7.2/calculateroute.json?apiKey=aygYkBd62u1r7MLha9I4tc6CUfWVJTgd-ZRhiUPpGhk&waypoint0=geo!{source_latitude},{source_longitude}&waypoint1=geo!{dest_latitude},{dest_longitude}&mode=fastest;truck;traffic:disabled'.format(
    source_latitude = o_lat,
    source_longitude = o_lon,
    dest_latitude = d_lat,
    dest_longitude = d_lon
    )
    
    
    response = requests.get(url_string)
    if response.status_code == 200:
        result = json.loads(response.text)
        manueuver_list = []
        try:
            for i in result['response']['route'][0]['leg'][0]['maneuver']:
                manueuver_list.append((i['position']['latitude'], i['position']['longitude']))
                dict_source_destination[(o_loc_zip, d_loc_zip)] = manueuver_list
        except:
            print("Exception for ", o_lat, o_lon, d_lat, d_lon)
        #print(manueuver_list)
    else:
        print("Failed for ", o_lat, o_lon, d_lat, d_lon)
    
        
# counter = 1

# df1.apply(lambda x: get_route(x.o_loc_zip,x.o_lat, x.o_lon,
#     x.d_loc_zip, x.d_lat, x.d_lon), axis=1)

#np.save('dictionary.npy', dict_source_destination)


In [9]:
dict_source_destination = np.load('dictionary_2.npy',allow_pickle='TRUE').item()

In [10]:
len(dict_source_destination)

133580

In [11]:
#Distance Calculation

# Using haversine logic get distance between two GPS location
def get_distance_in_miles(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)


    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance_in_km = R * c
    distance_in_miles = int(distance_in_km * 0.62137)
    if distance_in_miles == 0:
        return 1
    else:
        return distance_in_miles
    
def find_new_position_on_ratio(previous_distance, next_distance, between_mile_value,  previous_lat, previous_lon, next_lat, next_lon):
    new_lat =  ((next_lat - previous_lat) * (between_mile_value - previous_distance)  /(next_distance - previous_distance)) + previous_lat
    new_lon  =  ((next_lon - previous_lon) * (between_mile_value - previous_distance)  /(next_distance - previous_distance)) + previous_lon
    return (round(new_lat, 4), round(new_lon,4) )


In [15]:
# This module converts the route into charging station
# Always, we add the start and the end route
find_charging_nodes = {}

counter = 1

for o_zip, d_zip in dict_source_destination:
    print(o_zip, d_zip)
    manuever_list = dict_source_destination[(o_zip, d_zip)]
    
    if len(manuever_list) == 0:
        continue
    cumulative_distance = [0]
    previous_distance = 0
    
    for position, value in enumerate(manuever_list[:-1]):
        previous_distance += get_distance_in_miles(manuever_list[position][0], manuever_list[position][1], manuever_list[position + 1][0], manuever_list[position + 1][1])
        cumulative_distance.append(previous_distance)
    
    
    # Default including the start and the end location
    new_coordinate = OrderedDict()
    new_coordinate[manuever_list[0]] = 1
    
    
    between_mile_value = charging_station_distance
    for position, value in enumerate(cumulative_distance[:-1]):
        while value >= between_mile_value:

            previous_lat = manuever_list[position - 1][0]
            previous_lon = manuever_list[position - 1][1]
            previous_distance = cumulative_distance[position - 1]

            next_lat = manuever_list[position][0]
            next_lon = manuever_list[position][1]
            next_distance = cumulative_distance[position]

            #print(previous_distance)
            #print(next_distance)

            new_coordinate[find_new_position_on_ratio(previous_distance, next_distance, between_mile_value, previous_lat, previous_lon, next_lat, next_lon)] = 1
            between_mile_value += charging_station_distance
    
    new_coordinate[manuever_list[-1]] = 1

    find_charging_nodes[(o_zip, d_zip)] = list(new_coordinate.keys())
    

(76063, 76712)
('79601', '76801')
(37040, 42276)
('37128', '37128')
('28273', '29501')
('64701', '64086')
(62450, 63601)
('54409', '54166')
('75231', '75243')
(19311, 19709)
(22841, 21045)
('62035', '62626')
(97030, 97008)
(75020, 75235)
('28110', '28208')
(35811, 35768)
(49036, 48386)
('46819', '46818')
(61705, 61550)
('37411', '37160')
(55021, 55121)
('85353', '92324')
('02301', '03077')
('66115', '66067')
(70586, 70601)
(60014, 53916)
('78254', '76504')
('24012', '24013')
(85635, 85607)
(78130, 78251)
('34234', '34243')
(31015, 31750)
('61443', '61611')
('38843', '35570')
('19114', '08067')
('32832', '32765')
('12095', '13820')
(44333, 44667)
(34602, 30241)
('97477', '97217')
(15106, 15301)
('46168', '46221')
('32811', '32839')
(39339, 39601)
(3878, 3106)
(78154, 78154)
('63367', '62035')
(60608, 61362)
('43140', '43138')
(24112, 24540)
(80906, 80239)
(72034, 72209)
('32818', '33823')
('15401', '15401')
('64701', '65802')
('30004', '30009')
(76240, 74701)
(31533, 31701)
(75604, 7580

(2747, 4240)
('62206', '65270')
(72120, 72034)
('37421', '37862')
(77072, 77357)
(70808, 70506)
('70454', '70401')
(16881, 16412)
('01905', '01970')
('06256', '06385')
(85382, 85085)
('77575', '77523')
('73159', '72716')
('64701', '50022')
('95624', '95678')
(37355, 37885)
(29209, 29325)
('37813', '28779')
(44903, 44906)
('26354', '26554')
(75801, 77864)
(77901, 77535)
(85051, 85308)
('27610', '23970')
(29210, 29063)
(1532, 3077)
('40769', '40741')
('31707', '31793')
(75401, 76033)
('42104', '37043')
(68117, 68124)
(7047, 17901)
('77065', '78130')
(80918, 80915)
(47129, 42141)
(20186, 22942)
('60620', '61081')
('93312', '92507')
('75067', '73075')
('43068', '43219')
('03079', '01923')
('19709', '19947')
('38583', '40741')
('62821', '62450')
('91761', '91708')
('66046', '66749')
(48801, 49036)
('87701', '81101')
('56001', '56470')
('19311', '21030')
(61571, 61761)
('35055', '35023')
('53072', '53215')
(35094, 35125)
(85710, 85710)
('92881', '90280')
(29033, 29169)
('38652', '35630')
('7

('27604', '27609')
('48912', '46933')
('70815', '70455')
('84115', '84098')
('77611', '70570')
(27529, 28315)
(7663, 11554)
(32534, 36502)
('40741', '37087')
(43228, 43228)
('22554', '23185')
('38556', '37809')
('35007', '35207')
('85310', '85335')
(29070, 29728)
(60543, 52002)
(25570, 25570)
('75654', '75633')
('38344', '38242')
(18045, 18052)
(64154, 66067)
(17901, 19808)
(77356, 77301)
(45804, 48162)
('25403', '22841')
(75803, 76155)
(20735, 20754)
('43920', '44646')
('08332', '08360')
(48879, 46933)
('89030', '89149')
(23502, 23885)
('64037', '66106')
(72716, 74066)
(39345, 39350)
('61081', '61254')
('74015', '74429')
('90505', '92115')
(23885, 23970)
('75243', '76102')
('47274', '47933')
(27520, 27577)
(37160, 38401)
('67226', '67037')
('85035', '85037')
('14580', '14048')
(37090, 40741)
('76384', '72716')
(52655, 61270)
('45805', '43502')
('93635', '93249')
(75025, 75040)
(3079, 6260)
('55420', '55431')
('28262', '28205')
('93637', '93611')
(40033, 37072)
('45431', '45431')
(3242

('54409', '54966')
(34613, 34748)
('39540', '39540')
('34685', '34677')
(17402, 21237)
(62450, 62864)
(71822, 72143)
('72956', '64801')
('87031', '79934')
('23061', '23885')
('75220', '75115')
('62095', '62650')
('84660', '84029')
(34684, 34698)
('78539', '78542')
(35976, 35957)
('78251', '78254')
(65233, 65301)
('31313', '32063')
('01201', '12078')
('43050', '44654')
(77351, 77535)
('52302', '61270')
('14760', '16701')
('95832', '95757')
('67124', '66106')
('47304', '46901')
('78070', '78217')
('91744', '92801')
(61443, 61401)
('21601', '15522')
(30655, 30052)
('22841', '26508')
('65807', '65807')
('47394', '46819')
(13790, 16881)
('76302', '76308')
('47274', '46123')
(40391, 47274)
(19053, 19977)
(27407, 27406)
(63367, 62301)
(92881, 91761)
(75146, 75172)
(35150, 35242)
(35022, 30213)
(92324, 85704)
('43616', '48152')
(73069, 73114)
('43701', '43219')
(17870, 17901)
(81505, 84307)
('17078', '17111')
(1532, 1607)
('34476', '34481')
('65270', '63640')
(27028, 27302)
('60487', '60478')


('90745', '85326')
('40014', '40241')
(46113, 46221)
(44720, 44221)
('71291', '70570')
(77096, 77523)
(50613, 54751)
('77584', '77573')
('32641', '31533')
('29360', '29621')
(24273, 28704)
('17319', '17050')
(30241, 30078)
(31757, 31605)
('78748', '77474')
('98683', '98664')
('76548', '76504')
('78610', '76504')
('75062', '76266')
(45365, 45377)
(36426, 36109)
(29607, 29325)
('75904', '77028')
(37160, 37066)
('56001', '58301')
(60525, 60477)
('39571', '36544')
('03581', '03785')
(73750, 73703)
(60628, 61081)
('85204', '85193')
('34234', '34269')
(70791, 70760)
('76705', '79072')
('28906', '29650')
('52806', '52806')
(34239, 34269)
('46375', '61362')
('30214', '30241')
('76234', '75236')
(74352, 74344)
('92324', '84095')
(30721, 35055)
('76266', '76087')
('46706', '49321')
('76114', '75751')
('52040', '46327')
('83318', '84104')
(32771, 32746)
(32055, 32626)
('06088', '01119')
(72143, 72364)
(84032, 84737)
('32063', '35055')
(89434, 96097)
(17050, 18202)
('58203', '58301')
(16635, 16652

...

('23885', '20011')
('90703', '90670')
('50213', '64424')
('27537', '28451')
('70586', '70601')
(37067, 37015)
('44281', '44129')
('37334', '37091')
('62650', '61362')
(49120, 46901)
(4240, 2921)
('66508', '66067')
('44146', '44646')
(29575, 29464)
(72830, 71112)
('53132', '61362')
('80112', '80104')
('34224', '33781')
(49321, 46933)
('92027', '91942')
(31217, 30288)
(47240, 40741)
('02767', '03077')
('77055', '77449')
(36010, 36801)
('03077', '02767')
(49036, 49090)
(52641, 55431)
(89115, 84307)
(38114, 35055)
('14206', '18372')
('46933', '49090')
('46804', '46816')
('23885', '27909')
('72802', '71203')
(85711, 85710)
('53590', '53930')
(37127, 42103)
('46383', '46360')
(75603, 75765)
(75077, 75067)
(13815, 13748)
('29203', '29325')
('76105', '75067')
('54923', '54913')
('28348', '29577')
('30189', '30655')
('43015', '43230')
('02771', '02777')
(33442, 33428)
('73703', '72716')
('75051', '79072')
(95621, 95630)
('08077', '19804')
(34446, 34453)
(8094, 8094)
('26301', '26651')
(65355, 6

('65270', '65203')
('92583', '92545')
('22033', '20110')
(30038, 30315)
('21801', '21921')
('29706', '29624')
('84651', '84631')
('47586', '47170')
('95035', '95112')
(71241, 71107)
(72315, 63901)
('33330', '33322')
('87114', '80538')
(80538, 80123)
('87401', '87401')
('78666', '78109')
('35045', '35055')
('50022', '52802')
(8691, 13403)
(98930, 97338)
(60620, 60445)
(78599, 78542)
(76821, 76504)
('84737', '84104')
('95670', '95823')
(23139, 23901)
(24153, 15522)
('40831', '40729')
(61081, 60657)
('29456', '28150')
('34471', '32615')
('38614', '38652')
(28348, 29572)
(73160, 73069)
(37354, 40741)
('73127', '75115')
('75081', '75043')
('56001', '55912')
('90650', '90501')
('37813', '40771')
('83221', '83406')
('76017', '76262')
('46703', '49014')
('32003', '32063')
(2895, 18372)
(81501, 81401)
(71901, 75503)
(89706, 89705)
(79072, 73601)
('73114', '73013')
('31707', '31907')
('37160', '35758')
(33610, 33613)
('26105', '26301')
('22401', '22942')
(27284, 23885)
(17111, 17901)
(33458, 342

('85018', '85201')
(72450, 72455)
(73401, 76240)
('89074', '89014')
(28086, 28027)
(47586, 47112)
('47905', '47906')
(26105, 43123)
('60446', '46327')
(72716, 73107)
(78046, 78582)
(44676, 26105)
(92307, 92227)
('81212', '82009')
('17046', '22841')
(54843, 55746)
(98270, 98930)
('52641', '62704')
('85747', '85635')
('53901', '54923')
('06854', '18706')
('32771', '29303')
('74868', '79072')
(14580, 12095)
(40207, 40272)
('28052', '28054')
('76645', '79072')
(35594, 35570)
(49321, 49307)
('73005', '73114')
('66083', '52641')
(17078, 21030)
(46819, 48912)
('87701', '87571')
('89115', '89434')
(28273, 29710)
(61455, 62618)
('30349', '36010')
('63501', '64701')
('72117', '38111')
(76102, 76033)
('33837', '33619')
('77320', '76574')
('25260', '25313')
('22664', '22841')
('36330', '36305')
(67801, 73717)
('27560', '27616')
('17078', '17522')
('32063', '32757')
('48071', '48111')
(39601, 39111)
('47834', '62450')
(77406, 77420)
('36109', '36092')
(76549, 77523)
('62521', '60421')
(40769, 40403

('44060', '43160')
('59404', '84104')
(30180, 30349)
(67460, 67124)
('75052', '76033')
('92203', '92376')
('46327', '60439')
('70535', '77357')
('24592', '27215')
(49221, 43611)
(33990, 33919)
('75935', '75160')
('74501', '74447')
(28348, 28584)
('50021', '52802')
('32615', '34472')
(98597, 98421)
('29033', '29210')
(66701, 64147)
('67203', '67114')
('73064', '72716')
('75455', '75686')
('06416', '06489')
('44129', '44720')
(44266, 16125)
(83686, 83661)
('32824', '34981')
('23693', '23836')
('34481', '34476')
(63005, 63005)
(81005, 80216)
(84307, 81501)
(75074, 72756)
(29611, 28150)
(69101, 68117)
(63303, 63379)
(76306, 76262)
('33511', '33566')
(67204, 67203)
('38063', '38012')
('77023', '77535')
(74063, 74105)
('39218', '39208')
('14120', '14043')
(63401, 63552)
('62703', '62626')
(35160, 35954)
('27526', '22942')
('48723', '48604')
('60502', '61081')
('60108', '60101')
('97526', '96080')
(88240, 79415)
('98370', '98930')
(29360, 29334)
('04073', '04105')
('75007', '75056')
('66061',

('48315', '48038')
('78382', '76504')
(85032, 85051)
(75090, 76266)
(39564, 39401)
('15417', '16881')
('54452', '54729')
('29680', '35055')
('76712', '76710')
(43160, 43219)
('90620', '90620')
('75474', '76177')
('65340', '65233')
('17013', '17402')
(34234, 34269)
('21771', '21030')
(22560, 23693)
('23322', '23464')
('29303', '29334')
('49036', '49093')
(44202, 44062)
(30314, 30241)
(54615, 54017)
('08091', '08012')
(19365, 19530)
('33461', '33435')
('66067', '64037')
('28173', '27407')
(23456, 23707)
(23456, 23464)
(54660, 54409)
('53027', '53233')
('47909', '47933')
(56308, 56482)
(85193, 89183)
('92211', '92307')
('29728', '28461')
('71251', '71463')
('30329', '30180')
('67212', '66067')
('24151', '22942')
('21222', '15522')
('76504', '79701')
(90620, 92801)
('48509', '48532')
('39074', '38111')
(35968, 37160)
('76021', '75056')
(29728, 29526)
(62450, 62703)
('48912', '48471')
(20754, 20784)
('42437', '47610')
(95832, 95823)
(90805, 90501)
(61282, 52761)
('15601', '43123')
('40342',

In [16]:
len(find_charging_nodes)

133580

In [18]:
def is_route_present(o_zip, d_zip):
    global find_charging_nodes
    
    if (o_zip, d_zip) in find_charging_nodes:
        return True
    else:
        return False

In [20]:
df1['is_route_info_avail'] = df1.apply(lambda x : is_route_present(x.o_loc_zip, x.d_loc_zip), axis=1)
df_route = df1[df1.is_route_info_avail == True]
df_route.shape

(956610, 30)

In [22]:

def append_node_count(o_zip, d_zip):
    global find_charging_nodes
    
    if (o_zip, d_zip) in find_charging_nodes:
        for node_lat, node_lon in find_charging_nodes[(o_zip, d_zip)]:
            node_count[(node_lat, node_lon)] += 1


In [23]:

node_count = defaultdict(int)
df_route.apply(lambda x: append_node_count(x.o_loc_zip, x.o_loc_zip), axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
1009501    None
1009502    None
1009503    None
1009504    None
1009505    None
1009506    None
1009507    None
1009508    None
1009510    None
1009511    None
1009512    None
1009514    None
1009518    None
1009519    None
1009521    None
1009522    None
1009523    None
1009524    None
1009525    None
1009526    None
1009527    None
1009530    None
1009531    None
1009532    None
1009534    None
1009535    None
1009538    None
1009539    None
1009540    None
1009541    None
Length: 956610, dtype: o

In [28]:
node_count_list = []
for key in node_count:
    lat, lon = key
    node_count_list.append((node_count[key], lat, lon))
sorted_node_count_list = sorted(node_count_list, reverse=True)
print(sorted_node_count_list)
print(len(sorted_node_count_list))

[(6109, 40.7337632, -76.3187282), (6109, 40.7186794, -76.331509), (5345, 43.9900196, -90.4795575), (5293, 42.9895646, -74.4047644), (5293, 42.9886274, -74.3916635), (5292, 37.9829121, -78.2029366), (5292, 37.9780406, -78.2103554), (5286, 41.8151236, -89.6442211), (5286, 41.7795575, -89.7916138), (5152, 32.4063154, -97.4272016), (5152, 32.3408318, -97.4147308), (5112, 30.1740924, -95.2844776), (5112, 30.1522672, -95.2208877), (4961, 33.8089442, -83.6888051), (4961, 33.7987813, -83.7417924), (4950, 38.7499999, -88.0945372), (4950, 38.708462, -88.1224187), (4820, 34.7963082, -80.4102231), (4762, 39.5230687, -83.4566224), (4762, 39.5218181, -83.3964064), (4641, 30.2933514, -82.0809603), (4641, 30.2622905, -82.109233), (4494, 37.1466851, -84.1740704), (4494, 37.1131623, -84.0907163), (4304, 32.7010953, -85.3181505), (4304, 32.6372328, -85.421799), (4288, 34.4912852, -89.0667848), (4253, 40.4140988, -80.7376792), (4253, 40.3661084, -80.6693566), (4225, 35.4502453, -93.4604389), (4215, 39.535

In [27]:
for count,lat,lon in sorted_node_count_list:
    print(str(lat) +',' + str(lon))

40.7337632,-76.3187282
40.7186794,-76.331509
43.9900196,-90.4795575
42.9895646,-74.4047644
42.9886274,-74.3916635
37.9829121,-78.2029366
37.9780406,-78.2103554
41.8151236,-89.6442211
41.7795575,-89.7916138
32.4063154,-97.4272016
32.3408318,-97.4147308
30.1740924,-95.2844776
30.1522672,-95.2208877
33.8089442,-83.6888051
33.7987813,-83.7417924
38.7499999,-88.0945372
38.708462,-88.1224187
34.7963082,-80.4102231
39.5230687,-83.4566224
39.5218181,-83.3964064
30.2933514,-82.0809603
30.2622905,-82.109233
37.1466851,-84.1740704
37.1131623,-84.0907163
32.7010953,-85.3181505
32.6372328,-85.421799
34.4912852,-89.0667848
40.4140988,-80.7376792
40.3661084,-80.6693566
35.4502453,-93.4604389
39.5353746,-119.4794083
39.0694,-118.7139
37.9179,-117.7728
36.8283,-116.5869
36.0838226,-115.1488329
39.8966789,-83.1067657
39.8745812,-83.0427785
34.7366042,-97.247714
34.7046414,-97.2546167
38.6660943,-94.3668441
38.6118257,-94.3581498
34.2488265,-101.7042267
34.2120051,-101.7298472
29.7374828,-98.0628075
29.6

35.5601851,-77.0631403
35.5499999,-77.0500002
34.2580883,-91.9340964
34.219688,-91.8955918
33.9360595,-84.7298563
33.426311,-94.0567001
33.4089776,-94.0924029
30.5136416,-89.6627803
29.470429,-90.3254983
43.0379105,-88.1848955
43.0060077,-88.2309664
39.8937351,-80.1236762
39.0111852,-82.0356846
35.5499768,-88.813026
35.3007005,-90.7292867
34.6572026,-84.4876634
34.5386317,-83.973178
33.2409918,-92.6682186
33.1788152,-92.7219675
32.7675166,-89.1266307
32.7562991,-89.1046019
42.5869236,-72.2851488
42.5637615,-72.2857819
40.5853,-111.858716
37.859893,-82.045197
35.398495,-94.3630605
32.8091117,-83.7186569
32.1790438,-91.7289489
30.0741935,-90.4978609
44.9456739,-92.8968788
44.5496009,-122.9188328
44.5168591,-122.9022181
41.1280711,-78.7359837
34.8109424,-92.4859786
34.7611392,-79.485451
33.7261827,-93.6356778
33.6815071,-93.6155748
30.3362668,-89.1886999
44.5639479,-92.5828278
40.4882562,-79.8883724
40.4863036,-79.8689854
37.1495283,-121.6555488
37.1418199,-121.6524088
35.9310858,-89.8739